In [1]:
import os
from IPython.display import display, Image
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from scipy.stats import linregress
import math
from functools import reduce
import matplotlib
import argparse
from Bio import SeqIO, Entrez, pairwise2
Entrez.email = 'hongyingsun1101@gmail.com'
from Bio.SeqRecord import SeqRecord
import re, time
import os, sys, glob
import random
import uuid
# from skbio.tree import TreeNode
# from skbio import read
# from skbio.stats.distance import DistanceMatrix
# from skbio.stats.distance import DissimilarityMatrix

from scipy import stats
from ast import literal_eval
import sqlite3
# roc curve and auc score
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")



ModuleNotFoundError: No module named 'Bio'

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("all_data.csv", index_col=0)
score= pd.read_csv("score_merged.csv", index_col=0)

In [ ]:
reference ={'A':"RDP_10398", 'B':'RDP_5224', 'C':"RDP_1017", 'D':'RDP_92', 'E':'RDP_12'}

In [ ]:
def is_float(string):
  try:
    return float(string) and '.' in string  # True if string is a number contains a dot
  except ValueError:  # String is not a number
    return False

In [ ]:
def plot_pplacer(variable):
    fig, axes = plt.subplots(nrows=3, ncols=2)
    ax0, ax1, ax2, ax3, ax4, ax5 = axes.flatten()

    ax0.hist(df['A'+variable])
    ax0.set_title('A'+variable)

    ax1.hist(df['B'+variable])
    ax1.set_title('B'+variable)

    ax2.hist(df['C'+variable])
    ax2.set_title('C'+variable)

    ax3.hist(df['D'+variable])
    ax3.set_title('D'+variable)
    ax4.hist(df['E'+variable])
    ax4.set_title('E'+variable)


In [ ]:
def plotScatter(reference,community):
    fig, axes = plt.subplots(nrows=2, ncols=2)
    ax0, ax1, ax2, ax3 = axes.flatten()

    ax0.scatter(df[reference+'_adcl_log'], df[reference+community])
    ax0.set_title(reference+community+' vs '+ reference + '_adcl_log')
    
    ax1.scatter(df[reference+'_edpl'], df[reference+community])
    ax1.set_title(reference+community+' vs '+ reference + '_edpl')
    
    ax2.scatter(df[reference+'_mindistl'], df[reference+community])
    ax2.set_title(reference+community+' vs '+ reference + '_mindistl')
    
    ax3.scatter(df[reference+'_prichness'], df[reference+community])
    ax3.set_title(reference+community+' vs '+ reference + '_prichness')


In [ ]:
# plotScatter('A','0')

In [ ]:
# plotScatter('B','0')

In [ ]:
def plotScatterRef(variable,community):
    fig, axes = plt.subplots(nrows=3, ncols=2)
    ax0, ax1, ax2, ax3, ax4, ax5 = axes.flatten()

    ax0.scatter(df['A'+variable], df['A'+community])
    ax0.set_title('A' + community +' vs A' + variable)
    ax1.scatter(df['B'+variable], df['B'+community])
    ax1.set_title('B' + community +' vs B' + variable)
    
    ax2.scatter(df['C'+variable], df['C'+community])
    ax2.set_title('C' + community +' vs C' + variable)
    
    ax3.scatter(df['D'+variable], df['D'+community])
    ax3.set_title('D' + community +' vs D' + variable)  
    
    ax4.scatter(df['E'+variable], df['E'+community])
    ax4.set_title('E' + community +' vs E' + variable)   
    
    ax5.scatter(df['E'+variable], df['E'+community])
    ax5.set_title('E ' + community +'vs E' + variable)       


    
    
plotScatterRef('_adcl_log','0');

In [ ]:
    
# plotScatterRef('_edpl','99');

In [ ]:
cols=df.columns.tolist()
# cols[:20]

In [ ]:
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [ ]:
def plot_roc(data_X, class_label):
    trainX, testX, trainy, testy = train_test_split(data_X, class_label, test_size=0.3, random_state=1)
    model = RandomForestClassifier()
    model.fit(trainX, trainy)
    probs = model.predict_proba(testX)
    probs = probs[:, 1]
    auc = roc_auc_score(testy, probs)
    fpr, tpr, thresholds = roc_curve(testy, probs)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    print('optimal_threshold: %.2f' % optimal_threshold)
    print('AUC: %.2f' % auc)
    print( thresholds)
#     print( thresholds)
#     print('Model: ')
#     print(model)
    plot_roc_curve(fpr, tpr)

In [ ]:
def makeTable(headerRow,columnizedData,columnSpacing=2):
    """Creates a technical paper style, left justified table"""
    from numpy import array,max,vectorize

    cols = array(columnizedData,dtype=str)
    colSizes = [max(vectorize(len)(col)) for col in cols]

    header = ''
    rows = ['' for i in cols[0]]

    for i in range(0,len(headerRow)):
        if len(headerRow[i]) > colSizes[i]: colSizes[i]=len(headerRow[i])
        headerRow[i]+=' '*(colSizes[i]-len(headerRow[i]))
        header+=headerRow[i]
        if not i == len(headerRow)-1: header+=' '*columnSpacing

        for j in range(0,len(cols[i])):
            if len(cols[i][j]) < colSizes[i]:
                cols[i][j]+=' '*(colSizes[i]-len(cols[i][j])+columnSpacing)
            rows[j]+=cols[i][j]
            if not i == len(headerRow)-1: rows[j]+=' '*columnSpacing

    line = '-'*len(header)
    print(line)
    print(header)
    print(line)
    for row in rows: print(row)
    print(line)
header = ['AUROC','Categoroy']
cutoffs = ['0.9-1.0','0.8-0.9','0.7-0.8','0.6-0.7','0.5-0.6']
evalualtion = ['Very good','Good','Fair', 'Poor', 'Fail']
makeTable(header,[cutoffs,evalualtion])

In [ ]:
def plot_roc_microbiome(data_X, class_label, x, y, data_test=False):
    if(not data_test):
#         print("data_set is False")
        trainX, testX, trainy, testy = train_test_split(data_X, class_label, test_size=0.3, random_state=1)
        model = RandomForestClassifier()
        model.fit(trainX, trainy)
        probs = model.predict_proba(testX)
        probs = probs[:, 1]
        auc = roc_auc_score(testy, probs)
        fpr, tpr, thresholds = roc_curve(testy, probs)
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]
        print('optimal_threshold: %.2f' % optimal_threshold)
        print('AUC: %.2f' % auc)
        print('thresholds: ' + thresholds)

        plot_roc_curve(fpr, tpr)

    else:
        print("data_set is True")
        trainX, testX, trainy, testy = train_test_split(data_X, class_label, test_size=0.3, random_state=1)
        model = RandomForestClassifier()
        model.fit(trainX, trainy)
        probs1 = model.predict_proba(testX)
        probs1 = probs1[:, 1]
        auc1 = roc_auc_score(testy, probs1)
        fpr1, tpr1, thresholds1 = roc_curve(testy, probs1)
        optimal_idx1 = np.argmax(tpr1 - fpr1)
        optimal_threshold1 = thresholds1[optimal_idx1]
        print('AUC1: %.2f' % auc1)
        print('optimal_threshold1: %.2f' % optimal_threshold1)
        print(thresholds1)
        plot_roc_curve(fpr1, tpr1)
        
        probs2 = model.predict_proba(x)
        probs2 = probs2[:, 1]
        auc2 = roc_auc_score(y, probs2)
        fpr2, tpr2, thresholds2 = roc_curve(y, probs2)
        optimal_idx2 = np.argmax(tpr2 - fpr2)
        optimal_threshold2 = thresholds2[optimal_idx2]
        print('AUC2: %.2f' % auc2)
        print('optimal_threshold2: %.2f' % optimal_threshold2)
        print( thresholds2)
        plot_roc_curve(fpr2, tpr2)


In [ ]:
def plot_roc_curve_microbiome(pplacer_ref_list, pplacer_stats_list, community_list, cutoff_list, scoreOption=True):
    for (refIndex,pplacer_ref) in enumerate(pplacer_ref_list):
#     for refIndex in range(len(pplacer_ref_list)):
#         pplacer_ref = pplacer_ref_list[refIndex]
        for (statsIndex,pplacer_stats) in enumerate(pplacer_stats_list):
#         for statsIndex in range(len(pplacer_stats_list)):
#             pplacer_stats = pplacer_stats_list[statsIndex]
            for (communityIndex,community) in enumerate(community_list):
#             for communityIndex in range(len(community_list)):
#                 community = community_list[communityIndex]
                for (i, cutoff) in enumerate(cutoff_list):
#                 for i in range(len(cutoff_list)):
#                     cutoff=cutoff_list[i]
                    if(is_float(cutoff)):
                        cutoff_binary=float(cutoff)
                    else:
                        if(scoreOption):
                            cutoff_binary=float(df[pplacer_ref+community].describe().loc[[cutoff]])

                        else:
                            cutoff_binary = float(df[pplacer_ref+pplacer_stats].describe().loc[[cutoff]])
                    if(scoreOption):
                        mask = df[pplacer_ref+community] <=  cutoff_binary
                        df.loc[mask, pplacer_ref+community+'_binary'] = 1
                        mask = df[pplacer_ref+community] >cutoff_binary
                        df.loc[mask, pplacer_ref+community+'_binary'] = 0
                        df_binary = df[[pplacer_ref+pplacer_stats, pplacer_ref+community+'_binary']].dropna()
                        data_stats = df_binary[pplacer_ref+pplacer_stats].to_numpy().reshape(-1,1)
                        binary_label =  df_binary[pplacer_ref+community+'_binary'].to_numpy()
                        print(' The score cutoff '+ cutoff +' for Reference ' + pplacer_ref +' community ' + community   + ' with pplacer_stats '+ pplacer_stats[1:] + ': %.2f' % cutoff_binary )
                        plot_roc(data_stats,binary_label)
                    else:
                        mask = df[pplacer_ref+pplacer_stats] <=  cutoff_binary
                        df.loc[mask, pplacer_ref+pplacer_stats+'_binary'] = 1
                        mask = df[pplacer_ref+pplacer_stats] >cutoff_binary
                        df.loc[mask, pplacer_ref+pplacer_stats+'_binary'] = 0
                        df_binary = df[[pplacer_ref+community, pplacer_ref+pplacer_stats+'_binary']].dropna()
                        data_stats = df_binary[pplacer_ref+community].to_numpy().reshape(-1,1)
                        binary_label =  df_binary[pplacer_ref+pplacer_stats+'_binary'].to_numpy()
                        print(' The pplacer_stats_cutoff '+ cutoff +' for Reference ' + pplacer_ref +' community ' + community + ' pplacer_stats '  + pplacer_stats[1:]  + ': %.2f' % cutoff_binary )
                        plot_roc(data_stats,binary_label)
                

# different reference same pplacer stats same community to test different cutoffs  and different references for score

In [ ]:
# plot_roc_curve_microbiome(pplacer_ref_list = ['A','B','C','D','E'],pplacer_stats_list=['_adcl_log'],community_list=['A'],cutoff_list=['mean', 'min','25%','50%','75%'],scoreOption=False)

In [ ]:
df['E0'].describe()

# Different reference same pplacer stats same community to test different cutoffs  and different references for adcl_log

# Fitting on large reference and test on small reference datasets

In [ ]:
def plot_roc_curve_microbiome_test2(pplacer_ref_list, pplacer_stats_list, community_list, cutoff_list, test_data_list, scoreOption=True, testOption=False):
    for refIndex in range(len(pplacer_ref_list)):
        pplacer_ref = pplacer_ref_list[refIndex]
        for statsIndex in range(len(pplacer_stats_list)):
            pplacer_stats = pplacer_stats_list[statsIndex]
            for communityIndex in range(len(community_list)):
                community = community_list[communityIndex]
                for i in range(len(cutoff_list)):
                    cutoff=cutoff_list[i]
                    if(is_float(cutoff)):
                        cutoff_binary=float(cutoff)
                    else:
                        if(scoreOption):
                            cutoff_binary=float(df[pplacer_ref+community].describe().loc[[cutoff]])

                        else:
                            cutoff_binary = float(df[pplacer_ref+pplacer_stats].describe().loc[[cutoff]])
                    # no test situation, which is the default option
                    if (not testOption):

                        if(scoreOption):
                            mask = df[pplacer_ref+community] <=  cutoff_binary
                            df.loc[mask, pplacer_ref+community+'_binary'] = 1
                            mask = df[pplacer_ref+community] >cutoff_binary
                            df.loc[mask, pplacer_ref+community+'_binary'] = 0
                            df_binary = df[[pplacer_ref+pplacer_stats, pplacer_ref+community+'_binary']].dropna()
                            data_stats = df_binary[pplacer_ref+pplacer_stats].to_numpy().reshape(-1,1)
                            binary_label =  df_binary[pplacer_ref+community+'_binary'].to_numpy()
                            print(' The score cutoff '+ cutoff +' for Reference ' + pplacer_ref +' community ' + community   + ' with pplacer_stats '+ pplacer_stats[1:] + ': %.2f' % cutoff_binary )
                            # plot_roc(data_stats,binary_label)
                            plot_roc_microbiome(data_stats,binary_label,x=None,y=None,data_test=False)
                        else:
                            mask = df[pplacer_ref+pplacer_stats] <=  cutoff_binary
                            df.loc[mask, pplacer_ref+pplacer_stats+'_binary'] = 1
                            mask = df[pplacer_ref+pplacer_stats] >cutoff_binary
                            df.loc[mask, pplacer_ref+pplacer_stats+'_binary'] = 0
                            df_binary = df[[pplacer_ref+community, pplacer_ref+pplacer_stats+'_binary']].dropna()
                            data_stats = df_binary[pplacer_ref+community].to_numpy().reshape(-1,1)
                            binary_label =  df_binary[pplacer_ref+pplacer_stats+'_binary'].to_numpy()
                            print(' The pplacer_stats_cutoff '+ cutoff +' for Reference ' + pplacer_ref +' community ' + community + ' pplacer_stats '  + pplacer_stats[1:]  + ': %.2f' % cutoff_binary )
                            # plot_roc(data_stats,binary_label)
                            plot_roc_microbiome(data_stats,binary_label,x=None,y=None,data_test=False)
                    
                    # if there is test
                    else:
                        for j in range(len(test_data_list)):
                            test=test_data_list[j]
                            if(scoreOption):
                                mask = df[pplacer_ref+community] <=  cutoff_binary
                                df.loc[mask, pplacer_ref+community+'_binary'] = 1
                                mask = df[pplacer_ref+community] >cutoff_binary
                                df.loc[mask, pplacer_ref+community+'_binary'] = 0
                                df_binary = df[[pplacer_ref+pplacer_stats, pplacer_ref+community+'_binary']].dropna()
                                data_stats = df_binary[pplacer_ref+pplacer_stats].to_numpy().reshape(-1,1)
                                binary_label =  df_binary[pplacer_ref+community+'_binary'].to_numpy()
                                
                                mask_test = df[test+community] <=  cutoff_binary
                                df.loc[mask_test, test+community+'_binary'] = 1
                                mask_test = df[test+community] >cutoff_binary
                                df.loc[mask_test, test+community+'_binary'] = 0
                                df_binary = df[[test+pplacer_stats, test+community+'_binary']].dropna()
                                x = df_binary[test+pplacer_stats].to_numpy().reshape(-1,1)
                                y =  df_binary[test+community+'_binary'].to_numpy()

                                print(' The score cutoff '+ cutoff +' for Reference ' + pplacer_ref +' community ' + community   + ' with pplacer_stats '+ pplacer_stats[1:] + ' compared with test ' + test +': %.2f' % cutoff_binary )
                                plot_roc_microbiome(data_stats,binary_label,x,y,data_test=True)
                            else:

                                mask = df[pplacer_ref+pplacer_stats] <=  cutoff_binary
                                df.loc[mask, pplacer_ref+pplacer_stats+'_binary'] = 1
                                mask = df[pplacer_ref+pplacer_stats] >cutoff_binary
                                df.loc[mask, pplacer_ref+pplacer_stats+'_binary'] = 0
                                df_binary = df[[pplacer_ref+community, pplacer_ref+pplacer_stats+'_binary']].dropna()
                                data_stats = df_binary[pplacer_ref+community].to_numpy().reshape(-1,1)
                                binary_label =  df_binary[pplacer_ref+pplacer_stats+'_binary'].to_numpy()
                                
                                mask_test = df[test+pplacer_stats] <=  cutoff_binary
                                df.loc[mask_test, test+pplacer_stats+'_binary'] = 1
                                mask_test = df[test+pplacer_stats] >cutoff_binary
                                df.loc[mask_test, test+pplacer_stats+'_binary'] = 0
                                df_binary = df[[test+community, test+pplacer_stats+'_binary']].dropna()
                                x = df_binary[test+community].to_numpy().reshape(-1,1)
                                y =  df_binary[test+pplacer_stats+'_binary'].to_numpy()
                  
                                print(' The pplacer_stats_cutoff '+ cutoff +' for Reference ' + pplacer_ref +' community ' + community + ' pplacer_stats '  + pplacer_stats[1:] + ' compared with test ' + test  + ': %.2f' % cutoff_binary )
                                plot_roc_microbiome(data_stats,binary_label,x,y,data_test=True)

## Model from larger reference sets to fit data used small reference set.  Could be worse on both directions

In [ ]:
# plot_roc_curve_microbiome_test2(pplacer_ref_list = ['A'],pplacer_stats_list=['_adcl_log','_edpl','_prichness','_mindistl'],community_list=['0'],cutoff_list=['2.00'], test_data_list=['B','C','D','E'],testOption=True, scoreOption=True)

In [ ]:
# plot_roc_curve_microbiome_test2(pplacer_ref_list = ['A'],pplacer_stats_list=['_adcl_log'],community_list=['0'],cutoff_list=['-4.00'], test_data_list=['B','C','D','E'],testOption=True, scoreOption=False)

In [ ]:
# plot_roc_curve_microbiome_test2(pplacer_ref_list = ['A'],pplacer_stats_list=['_adcl_log'],community_list=['0'],cutoff_list=['25%'], test_data_list=['B','C'],testOption=True, scoreOption=False)

In [ ]:
# plot_roc_curve_microbiome_test2(pplacer_ref_list = ['B'],pplacer_stats_list=['_adcl_log'],community_list=['0'],cutoff_list=['25%'], test_data_list=['A','C'],testOption=True, scoreOption=False)

In [ ]:
# print("the head for df is {}".format(df.head)+ " the columns of the df is {}".format(df.columns))
#       

In [ ]:
# df['A0'].describe(), df['B0'].describe(), df['C0'].describe(), df['D0'].describe(),df['E0'].describe()


In [ ]:

# for community in ['A','B','C','D','E']:
#     for i in range(10):
#             print(df[community+str(i)].describe())


In [ ]:
df_0 = df

In [ ]:
# plot_pplacer('90')

In [ ]:
# plotScatter('B','0')

In [ ]:
# plotScatterRef('_adcl_log','0')

In [ ]:
# plot_pplacer('_adcl_log')

In [ ]:
df['A_adcl_log'].describe()



In [ ]:
# plot_pplacer('0')

In [ ]:
df['A0'].describe()


In [ ]:
df1 = df[(df['A0']>10)]

In [ ]:
df1['A0'].describe()

In [ ]:
99/605

In [ ]:
df1['B0'].describe()

In [ ]:
df2=df[['seqID','A0','B0', 'C0','D0','E0']].dropna()

In [ ]:
df3 = df2[(df2['A0']>10) & (df2['B0']>10)  & (df2['C0']>10)  & (df2['D0']>10) & (df2['E0']>10)]

In [ ]:
# df2.describe(), df3.describe()

In [ ]:
df3

In [ ]:
2/605

In [ ]:
dfc90 = df[(df['A90']>10) & (df['B90']>10)  & (df['C90']>10)  & (df['D90']>10) & (df['E90']>10)]

In [ ]:
dfc90['B0'].describe()

In [ ]:



df[(df.community=='CC11CM0')]['C_adcl_log'].dropna().describe()

In [ ]:
plot_pplacer('_prichness')

In [ ]:
df['A_prichness'].describe()

In [ ]:
df[df.A0>10].A0.count()

In [ ]:
df[df.A0>10].A0.count()/df.A0.count()

In [ ]:

# df.head()

In [ ]:
df.A_adcl.count()

In [ ]:
d={"a":1, "b":2}

In [ ]:
d

In [ ]:
dd = pd.Series(d, name='score')

In [ ]:

dd.index.name="community"

In [ ]:
dd.reset_index()

In [ ]:
"CC11CM"+str(0)

In [ ]:
c0=df['A0'][df['community']=='CC11CM0']

In [ ]:
per=c0[c0>10].count()/c0.count()

In [ ]:
def generateScore(stats, referenceID,scorecutoff,statscutoff):
    d1={}
    d2={}
    for i in range(100):
        values = df[referenceID+str(i)][df.community=='CC11CM'+str(i)]
        statsvalues = df[stats][df['community']=='CC11CM'+str(i)]
        d1['CC11CM'+str(i)] = values[values>scorecutoff].count()/values.count()
        d2['CC11CM'+str(i)] =statsvalues[statsvalues>statscutoff].count()/statsvalues.count()
    d1=pd.Series(d1, name=referenceID)
    d1.index.name='community'
    d1=d1.reset_index()
    d2=pd.Series(d2, name=stats)
    d2.index.name='community'
    d2=d2.reset_index()
    dt = pd.concat([d1,d2], axis=1)
#     dt=dt.set_index('community')
    return (dt)

In [ ]:
dt=generateScore('A_adcl', 'A', 10, 0.001)

In [ ]:
plt.scatter(dt.A, dt.A_adcl)

In [ ]:

t=[]
for referenceID in ['A','B','C','D','E']:
    t.append(generateScore('A_adcl', referenceID, 10, 0.001))

In [ ]:


t[0].head()

In [ ]:
tt = pd.concat([t[0],t[1],t[2],t[3],t[4]], axis=1)

In [ ]:
def generateScoreu(stats, referenceID,scorecutoff,statscutoff):
    d1={}
    d2={}
    for i in range(100):
        values = df[referenceID+str(i)][df.community=='CC11CM'+str(i)]
        statsvalues = df[referenceID+stats][df['community']=='CC11CM'+str(i)]
        d1['CC11CM'+str(i)] = values[values>scorecutoff].count()/values.count()
        d2['CC11CM'+str(i)] =statsvalues[statsvalues>statscutoff].count()/statsvalues.count()
    d1=pd.Series(d1, name=referenceID)
    d1.index.name='community'
    d1=d1.reset_index()
    d2=pd.Series(d2, name=referenceID+stats)
    d2.index.name='community'
    d2=d2.reset_index()
    dt = pd.concat([d1,d2], axis=1)
    dt=dt.loc[:, ~dt.columns.duplicated()]
    dt=dt.set_index('community')
    return (dt)

In [ ]:
dtu=generateScoreu('_adcl', 'A', 10, 0.001)

In [ ]:
plt.scatter(dtu.A, dtu.A_adcl)

In [ ]:
t=[]
statsdir= {'_adcl':0.0001, '_edpl':0,'_prichness':10,'_mindistl':0.05}
for stats in statsdir.keys():

    for referenceID in ['A','B','C','D','E']:
        t.append(generateScoreu(stats, referenceID, 10, statsdir[stats]))

In [ ]:
ttt = pd.concat([t[0],t[1],t[2],t[3],t[4],t[5],t[6],t[7],t[8],t[9],t[10],t[11],t[12],t[13],t[14],t[15],t[16],t[17],t[18],t[19]], axis=1)
ttt=ttt.loc[:, ~ttt.columns.duplicated()]

In [ ]:
# ttt.describe()

In [ ]:
ttt.to_csv("community-based.csv")

In [ ]:

plt.hist(df.A_mindistl)

In [ ]:

dp =pd.read_csv("community-based.csv", index_col=0)

In [ ]:

dp.describe()

In [ ]:


for score in ['A','B','C','D','E'][0:1]:
    for stats in ['_adcl', '_edpl','_prichness','_mindistl'][0:1]:
        plt.scatter(dp[score], dp[score+stats], label=score + ' vs '+ score+stats)
        plt.legend(loc='upper left')
        plt.show

In [ ]:
for score in ['A','B','C','D','E'][0:1]:
    for stats in ['_adcl', '_edpl','_prichness','_mindistl'][0:5]:
        plt.scatter(dp[score], dp[score+stats], label=score + ' vs '+ score+stats)
        plt.legend(loc='upper left')
        plt.show

In [ ]:
for score in ['A','B','C','D','E']:
    for stats in ['_adcl', '_edpl','_prichness','_mindistl']:
        plt.scatter(dp[score], dp[score+stats], label=score + ' vs '+ score+stats)
        plt.legend(loc='upper left')
        plt.show

In [ ]:
# dp.describe()

# Bray-Curtis distance


In [ ]:
>>> from scipy.spatial import distance
>>> distance.braycurtis([1, 0, 0], [0, 1, 0])
1.0
>>> distance.braycurtis([1, 1, 0], [0, 1, 0])
0.33333333333333331

In [ ]:
distance.braycurtis([1, 0, 0], [0, 1, 0])

In [ ]:
>>> distance.braycurtis([1, 1, 0], [0, 1, 0])

# Association between Bray-Curtis Distance and Sensitivity 

In [ ]:
bcd = pd.read_csv("Bray-Curtis-Distance-HS.csv", index_col=0)
sensitivity = pd.read_csv("alphaDiversity_phyloEntropy.csv", index_col=0)

In [ ]:
 bcd.head(),sensitivity.head()

In [ ]:
# bcd-sen = pd.merge(bcd, sensitivity, left_index=True, right_index=True)
bcdsen = pd.concat([bcd, sensitivity], axis=1)

In [ ]:
bcdsen.head()

In [ ]:
# df_new = df.rename(columns={'A': 'a'}, index={'ONE': 'one'})
bcdsen = bcdsen.rename(columns={'rdp10398':'bcd_rdp10398', 'rdp10398':'bcd_rdp10398','rdp5224':'bcd_rdp5224','rdp1017':'bcd_rdp1017','rdp92':'bcd_rdp92','rdp12':'bcd_rdp12','RDP_10398':'sensitivity_rdp10398','RDP_5224':'sensitivity_rdp5224','RDP_1017':'sensitivity_rdp1017','RDP_92':'sensitivity_rdp92','RDP_12':'sensitivity_rdp12'})

In [ ]:
bcdsen.head()

In [ ]:
# objects = ('Python', 'C++', 'Java', 'Perl', 'Scala', 'Lisp')
# y_pos = np.arange(len(objects))
# performance = [10,8,6,4,2,1]
# plt.bar(y_pos, performance, align='center', alpha=0.5)
# plt.xticks(y_pos, objects)
# plt.ylabel('Usage')
# plt.title('Programming language usage')
# plt.show()

In [ ]:
# objects=('bcd_rdp10398','bcd_rdp5224','bcd_rdp1017','bcd_rdp92','bcd_rdp12','sensitivity_rdp10398','sensitivity_rdp5224','sensitivity_rdp1017','sensitivity_rdp92','sensitivity_rdp12)
# plt.bar(bcdsen.bcd_rdp10398, )

In [ ]:
#         plt.scatter(bcdsen['bcd_rdp10398'], bcdsen['sensitivity_rdp10398'], label='BCD vs Sensitivity' )
#         plt.legend(loc='upper left')
#         plt.show

In [ ]:

# plt.scatter(bcdsen['bcd_rdp5224'], bcdsen['sensitivity_rdp5224'], label='BCD vs Sensitivity' )
# plt.legend(loc='upper left')
# plt.show

In [ ]:
# plt.scatter(bcdsen['bcd_rdp12'], bcdsen['sensitivity_rdp12'], label='BCD vs Sensitivity' )
# plt.legend(loc='upper left')
# plt.show

In [ ]:
# for refindex in ['10398', '5224','1017','92','12']:
#     plt.scatter(bcdsen['bcd_rdp'+refindex], bcdsen['sensitivity_rdp' + refindex ], label='RDP'  + refindex)
#     plt.legend(loc='lower right')
#     plt.show;
#     plt.savefig('bcd-sen.png')

<!-- ##  x axis is the Bray-Curtis distance, and y axis is the sensitivity in percent. Each dot is one community. Different colors indicate data using different references. -->

# Exploring data

# Adcl 

# Adcl plot all sequences in 5 reference sets

In [ ]:
plot_pplacer('_adcl')

In [ ]:
def plot_pplacer_cutoff(variable, cutoff):
    
    fig, axes = plt.subplots(nrows=3, ncols=2)
#     axes[i].set_ylim(0,1000)
    ax0, ax1, ax2, ax3, ax4, ax5 = axes.flatten()

    ax0.hist(df['A'+variable][df['A'+variable]>cutoff])
    ax0.set_title('A'+variable + ' larger than the cutoff' )

    ax1.hist(df['B'+variable][df['B'+variable]>cutoff])
    ax1.set_title('B'+variable)

    ax2.hist(df['C'+variable][df['C'+variable]>cutoff])
    ax2.set_title('C'+variable)

    ax3.hist(df['D'+variable][df['D'+variable]>cutoff])
    ax3.set_title('D'+variable)
    ax4.hist(df['E'+variable][df['E'+variable]>cutoff])
    ax4.set_title('E'+variable)

# Plot adcl with bad values larger than 0.001

In [ ]:
plot_pplacer_cutoff('_adcl', 0.001)

# Plot adcl with bad values larger than 0.15

In [ ]:
plot_pplacer_cutoff('_adcl', 0.15)

In [ ]:
stats.percentileofscore(df.A_adcl,0.001),stats.percentileofscore(df.B_adcl,0.001),stats.percentileofscore(df.C_adcl,0.001),stats.percentileofscore(df.D_adcl,0.001),stats.percentileofscore(df.E_adcl,0.001)

In [ ]:
stats.percentileofscore(df.A_adcl,0.15),stats.percentileofscore(df.B_adcl,0.15),stats.percentileofscore(df.C_adcl,0.15),stats.percentileofscore(df.D_adcl,0.15),stats.percentileofscore(df.E_adcl,0.15)

# edpl

In [ ]:
from scipy.stats import median_test
from scipy.stats import skew
# stat, p, med, tbl = median_test(g1, g2, g3)

In [ ]:
plot_pplacer('_edpl')

In [ ]:
plot_pplacer('_prichness')

In [ ]:
df.A_prichness.describe()

In [ ]:
stat, p, med, tbl = median_test(df['A_edpl'], df['B_edpl'], df['C_edpl'],df['D_edpl'],df['E_edpl'])

In [ ]:
stat,p,med,tbl

In [ ]:
stats.scoreatpercentile(df.A_edpl,95),stats.scoreatpercentile(df.B_edpl,95),stats.scoreatpercentile(df.C_edpl,95),stats.scoreatpercentile(df.D_edpl,95),stats.scoreatpercentile(df.E_edpl,95)

In [ ]:
stat, p, med, tbl = median_test(df['A_edpl'][df['A_edpl']>0.09], df['B_edpl'], df['C_edpl'],df['D_edpl'],df['E_edpl'])

In [ ]:
med

In [ ]:
# stat is The default is Pearson’s chi-squared statistic.
# tbl: contingency table is the number of counts above (first) or below (second) the median
# median is the grand median of all the data

In [ ]:
stat_prichness, p_prichness, med_prichness, tbl_prichness = median_test(df['A_prichness'], df['B_prichness'], df['C_prichness'],df['D_prichness'],df['E_prichness'])

In [ ]:
stat_prichness, p_prichness, med_prichness, tbl_prichness

In [ ]:
skew(df.A_prichness),skew(df.B_prichness),skew(df.C_prichness),skew(df.D_prichness),skew(df.E_prichness)

In [ ]:
skew(df.A_edpl),skew(df.B_edpl),skew(df.C_edpl),skew(df.D_edpl),skew(df.E_edpl)

In [ ]:
from scipy import stats
#  stats.percentileofscore([1, 2, 3, 4], 3)

In [ ]:
stats.percentileofscore(df.A_prichness,3),stats.percentileofscore(df.B_prichness,3),stats.percentileofscore(df.C_prichness,3),stats.percentileofscore(df.D_prichness,3),stats.percentileofscore(df.E_prichness,3)

In [ ]:
stats.percentileofscore(df.A_prichness,10),stats.percentileofscore(df.B_prichness,10),stats.percentileofscore(df.C_prichness,10),stats.percentileofscore(df.D_prichness,10),stats.percentileofscore(df.E_prichness,10)

In [ ]:
stats.scoreatpercentile(df.A_prichness,25),stats.scoreatpercentile(df.B_prichness,25),stats.scoreatpercentile(df.C_prichness,25),stats.scoreatpercentile(df.D_prichness,25),stats.scoreatpercentile(df.E_prichness,25)

In [ ]:
stats.scoreatpercentile(df.A_prichness,75),stats.scoreatpercentile(df.B_prichness,75),stats.scoreatpercentile(df.C_prichness,75),stats.scoreatpercentile(df.D_prichness,75),stats.scoreatpercentile(df.E_prichness,75)

In [ ]:
aa=stats.scoreatpercentile(df.A_prichness,75),stats.scoreatpercentile(df.B_prichness,75),stats.scoreatpercentile(df.C_prichness,75),stats.scoreatpercentile(df.D_prichness,75),stats.scoreatpercentile(df.E_prichness,75)